In [ ]:
import numpy as np
import pandas as pd
import joblib
from datetime import datetime

In [ ]:
from arthurai import ArthurAI
from arthurai.client.apiv3 import InputType, OutputType, Stage, AttributeCategory, AttributeBin, ValueType

## Set up connection to API

In [ ]:
URL = "app.arthur.ai"
ACCESS_KEY = ""

connection = ArthurAI(url=URL, access_key=ACCESS_KEY, client_version=3)

In [ ]:
model = connection.model(partner_model_id="MEPS_drift_1",
                         input_type=InputType.Tabular,
                         output_type=OutputType.Multiclass,
                         is_batch=True)

# uncomment the below if you want to get the same model object that you have already created
# model = connection.get_model('MEPS_drift_1', id_type='partner_model_id')

## Set up ArthurModel
Everything under this header is *necessary* for an `ArthurModel` to be created; additional functionality is not 
possible until after `model.save()` has been successfully called.

*Some context about this dataset:*

- label: `UTILIZATION`, where 1: >10 visits, 0: <10 visits

- protected attribute: `RACE`, where 1: `White`, 0: `Non-White`

In [ ]:
# train_data_full contains the X's and the Y's
all_data = pd.read_parquet('data/fulldata_train.parquet')

In [ ]:
# quick summary of what all_data looks like
all_data.head()

In [ ]:
all_data.columns.tolist()[-3:]

In [ ]:
# add the attributes used for training to the model and set them to ModelPipelineInput
# note that attribute names need to contain only letters, numbers, and underscores, and cannot begin with a number
train_x = all_data.drop(columns=['RACE','p_0', 'p_1', 'gt'])
model.from_dataframe(train_x, Stage.ModelPipelineInput)

In [ ]:
# gt_<>_utilization will refer to the ground truth values (i.e. true labels), while
# pred_<>_utilization will refer to the predicted outputs of your model. 
pred_to_ground_truth_map = {
    "pred_high_utilization": "gt_high_utilization",
    "pred_low_utilization": "gt_low_utilization"
}

# add the ground truth and predicted attributes to the model
model.add_binary_classifier_output_attributes('pred_high_utilization', pred_to_ground_truth_map)

In [ ]:
# add bias monitoring - this cannot be done after the model is saved
# to monitor for bias, 'RACE' must be an int (or take on discrete string values)
all_data['RACE'] = all_data['RACE'].astype(int)
sens_x = pd.DataFrame(data={'RACE': all_data['RACE']})
model.from_dataframe(sens_x, Stage.NonInputData)

model.get_attribute('RACE', stage=Stage.NonInputData).monitor_for_bias=True

In [ ]:
# check all the attributes loaded to the model. note that even though you passed in your "real" training
# data, currently the model has no reference to the actual datapoints -- just the properties of the 
# attributes. you will need to set reference data (for data drift detection) later
model.review()

In [ ]:
# your model is not uploaded to Arthur until you call model.save()
model.save()
# model.update()

## Adding additional functionality

Setting reference data and monitoring for bias adds to the functionality of the `ArthurModel` that you've created. 

### Setting reference data

Now that we've saved the model, we can set reference data.
The reference data df must have a column for each `ModelPipelineInput` and each `NonInput`. Optionally, it 
can contain predicted value and ground truth columns; this will enable the calculation of data drift 
on output attributes. In this model, our ground truth columns are `gt_high_utilization` and `gt_low_utilization`,
and our predicted value columns are `pred_high_utilization` and `pred_low_utilization`. 

We need to rename and update the columns in `all_data` to match these. 

In [ ]:
# ['p_0', 'p_1', 'gt']
all_data = all_data.rename(columns={'p_0': 'pred_low_utilization', 'p_1': 'pred_high_utilization'})
gt = pd.get_dummies(all_data['gt'], prefix='gt', dtype='int')
gt = gt.rename(columns={'gt_0.0': 'gt_low_utilization', 'gt_1.0': 'gt_high_utilization'})
all_data = all_data.drop(columns=['gt'])
all_data = all_data.merge(gt, left_index=True, right_index=True)

In [ ]:
all_data.dtypes

In [ ]:
model.set_reference_data(data=all_data) # note: need to explicitly specify the data= argument

### send batches of inferences

We do not need to load/expose the actual predictive model to send inferences to `arthur_model`. `x_shift` is a directory holding batched data. Each of the parquet files is a dataframe with the following columns:

- sensitive attribute, `RACE`
- all attributes used for training
- `p_0`: the predicted probability of this example being in the 0 class - *given by your model*
- `p_1`: the predicted probability of this example being in the 1 class - *given by your model*
- `gt`: the ground truth label. 

In [ ]:
%ls x_shift 

##### batch inference format
(df option)
Your batch inferences must have the dummied prediction columns that you created earlier; 
if you don't have ground truth labels at this time, you don't need to send them now.
When sending batch inferences, you must add the following additional columns: 
- `batch_id` - string
- `partner_inference_id`- string
- `inference_timestamp` - datetime.datetime

In [ ]:
for i in range(4): # sending inferences without ground truth
    alldata = pd.read_parquet('data/x_shift/fulldata_'+str(i) +'.parquet')
    tosend = alldata.rename(columns={'p_0': 'pred_low_utilization', 'p_1': 'pred_high_utilization'})
    tosend['RACE'] = tosend['RACE'].astype(int)
    
    tosend['batch_id'] = [str(i)]*len(tosend)
    tosend['partner_inference_id'] = [str(np.random.randint(10000))]*len(tosend)
    tosend['inference_timestamp'] = [(datetime.now())]*len(tosend)
    
    model.send_batch_inferences(data=tosend)

In [ ]:
for i in range(4): # adding ground truth later
    alldata = pd.read_parquet('data/x_shift/fulldata_'+str(i) +'.parquet')
    tosend = pd.get_dummies(alldata['gt'], prefix='gt')
    tosend = tosend.rename(columns={'gt_0.0': 'gt_low_utilization', 'gt_1.0': 'gt_high_utilization'})

    tosend['batch_id'] = [str(i)]*len(tosend)
    tosend['partner_inference_id'] = [str(np.random.randint(10000))]*len(tosend)
    tosend['inference_timestamp'] = [(datetime.now())]*len(tosend)
    
    model.send_batch_inferences(data=tosend)

In [ ]:
for i in range(4): # adding gt and infs at the same time
    alldata = pd.read_parquet('data/x_shift/fulldata_'+str(i+4) +'.parquet')
    tosend = alldata.rename(columns={'p_0': 'pred_low_utilization', 'p_1': 'pred_high_utilization'})
    tosend['RACE'] = tosend['RACE'].astype(int)
    
    gts = pd.get_dummies(alldata['gt'], prefix='gt')
    gts = gts.rename(columns={'gt_0.0': 'gt_low_utilization', 'gt_1.0': 'gt_high_utilization'})
    
    tosend = tosend.merge(gts, left_index=True, right_index=True)

    tosend['batch_id'] = [str(i+4)]*len(tosend)
    tosend['partner_inference_id'] = [str(np.random.randint(10000))]*len(tosend)
    tosend['inference_timestamp'] = [(datetime.now())]*len(tosend)
    
    model.send_batch_inferences(data=tosend)